In [1]:
import os
import pandas as pd
import sys,os
sys.path.append('../../../')
from env.trading_env import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl import config_tickers
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR

check_and_make_directories([TRAINED_MODEL_DIR])

/home/youyou/anaconda3/envs/finrl/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/tmp/ipykernel_30162/2005910646.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
train = pd.read_csv('../train_data.csv')

# If you are not using the data generated from part 1 of this tutorial, make sure 
# it has the columns and index in the form that could be make into the environment. 
# Then you can comment and skip the following two lines.
train = train.set_index(train.columns[0])
train.index.names = ['']

In [3]:
train

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
,,,,,,,,,,,,,,,,,,
0,2009-01-02,AAPL,3.067143,3.251429,3.041429,2.747390,746015200.0,4.0,0.000000,2.969345,2.641386,100.000000,66.666667,100.000000,2.747390,2.747390,39.189999,0.000000
0,2009-01-02,AMGN,58.590000,59.080002,57.750000,42.737888,6547900.0,4.0,0.000000,2.969345,2.641386,100.000000,66.666667,100.000000,42.737888,42.737888,39.189999,0.000000
0,2009-01-02,AXP,18.570000,19.520000,18.400000,15.144919,10955700.0,4.0,0.000000,2.969345,2.641386,100.000000,66.666667,100.000000,15.144919,15.144919,39.189999,0.000000
0,2009-01-02,BA,42.799999,45.560001,42.779999,33.941097,7010200.0,4.0,0.000000,2.969345,2.641386,100.000000,66.666667,100.000000,33.941097,33.941097,39.189999,0.000000
0,2009-01-02,CAT,44.910000,46.980000,44.709999,30.950012,7117200.0,4.0,0.000000,2.969345,2.641386,100.000000,66.666667,100.000000,30.950012,30.950012,39.189999,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2892,2020-06-30,UNH,288.570007,296.450012,287.660004,280.751923,2932900.0,1.0,-0.019008,296.506862,264.629827,52.413034,-26.031434,1.802109,280.989918,274.142010,30.430000,12.918687
2892,2020-06-30,V,191.490005,193.750000,190.160004,188.478638,9040100.0,1.0,1.036367,196.397044,182.850253,53.021026,-51.665655,3.333755,189.217591,179.526374,30.430000,12.918687
2892,2020-06-30,VZ,54.919998,55.290001,54.360001,44.822735,17414800.0,1.0,-0.388920,47.973949,43.356940,48.097047,-51.451270,6.514530,45.388063,45.790727,30.430000,12.918687


In [4]:
stock_dimension = len(train.tic.unique())

rebalancing_period=10

# state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
state_space = 1 + stock_dimension + (stock_dimension + len(INDICATORS)*stock_dimension)*rebalancing_period

print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 29, State Space: 2640


In [5]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4,
    
    "rebalancing_period":rebalancing_period,
    "model_name": 'PPO'

}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [6]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 256,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

tmp_path = RESULTS_DIR + '/ppo'
new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
# Set new logger
model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 256}
Using cuda device
Logging to results/ppo


In [7]:
trained_ppo = agent.train_model(model=model_ppo, 
                             tb_log_name='ppo',
                             total_timesteps=200#000
                             ) 

Episode: 2
============save===========
Episode: 3
============save===========
Episode: 4
============save===========
Episode: 5
============save===========
Episode: 6
============save===========
Episode: 7
============save===========
Episode: 8
============save===========
-----------------------------------
| time/              |            |
|    fps             | 281        |
|    iterations      | 1          |
|    time_elapsed    | 7          |
|    total_timesteps | 2048       |
| train/             |            |
|    reward          | -2.8100932 |
-----------------------------------


In [8]:
trained_ppo.save(TRAINED_MODEL_DIR + "/agent_ppo") 